In [1]:
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
cascPath = "haarcascade_frontalface_default.xml"
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)


In [24]:
## read dataset1
full_face = []
full_labels = []
all_images = os.listdir('yalefaces')
for j in range(1,16):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images if filename.startswith('subject%.2d'%j)]
#     img_list = glob.glob('yalefaces/subject%.2d*'%j)
    face1 = np.zeros((len(img_list),32,32))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(os.path.join('yalefaces',img))
        I = cv2.fastNlMeansDenoising(I, None, 9, 13)
        I = cv2.GaussianBlur(I, (5,5), 0)
        I = cv2.equalizeHist(I)
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        try : 
            x,y,h,w = faces[0]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])     
        except : 
            x,y,h,w=[0,0,479,639]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])   
        
    labels = (j-1)*np.ones((11,))
    full_face.append(face1)
    full_labels.append(labels)
full_face = np.stack(full_face)    # contains the images of all faces
full_labels = np.stack(full_labels)   # contains the ID of all the faces


In [25]:
full_face = full_face/255 #normalise data

In [26]:
full_face.shape

(15, 11, 32, 32)

In [44]:
#similar 1 dissimilar 0
pairs = []
labels = []
similarity = []
for i in range(0,15):
    for j in range(50):
        a = np.random.randint(0,11,2)
        labels.append([i,i])
        pairs.append(a)
        similarity.append(1)

In [45]:
#similar 1 dissimilar 0
for i in range(0,500):
        b = np.random.randint(0,15,2)
        a = np.random.randint(0,11,2)
        labels.append(b)
        pairs.append(a)
        if(b[0]==b[1]):
            similarity.append(1)
        else:
            similarity.append(0)

In [46]:
trainX = np.zeros((len(similarity),32,32))

In [62]:
X_train = np.zeros((len(similarity),32*32))
for i in range(len(similarity)):
    a = labels[i]
    b = pairs[i]
    face1 = full_face[a[0],b[0],:,:]
    face2 = full_face[a[1],b[1],:,:]
    face = face1-face2
    face = np.multiply(face,face)
    trainX[i,:,:]=face
    X_train[i,:]=trainX[i,:,:].ravel()

In [65]:
X_train.shape

(1250, 1024)

In [69]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 5), random_state=1)
clf.fit(X_train,similarity)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [70]:
clf.score(X_train,similarity)

0.9992

In [72]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,similarity)
lr.score(X_train,similarity)

0.9336

In [73]:
import h5py
h5f = h5py.File('test.h5','r')
a = list(h5f.keys())
## read the dataset
full_face = []
full_labels = []
img_list=[h5f[i] for i in a]
tface1 = np.zeros((len(img_list),32,32))
for k,img in enumerate(img_list):
    I = np.array(img)#scipy.misc.imread(os.path.join('yalefaces',img))
    #print(type(I))
    I = cv2.fastNlMeansDenoising(I, None, 9, 13)
    I = cv2.GaussianBlur(I, (5,5), 0)
    I = cv2.equalizeHist(I)
    faces = faceCascade.detectMultiScale(
        I,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))
    #print(k)
    #print(faces)
    try : 
        x,y,h,w = faces[0]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])
    except : 
        x,y,h,w=[0,0,111,91]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])

In [74]:
tface1.shape

(120, 32, 32)

In [75]:
tface1 = tface1/255

In [76]:
test_pairs = np.genfromtxt('image_pairs.csv', dtype=None, delimiter=',', names=True)

In [83]:
euc=[]

for i in range(3540):#(3540):
    x1 = np.zeros((32,32))
    x0 = np.zeros((32,32))
    ids = test_pairs[i][0]
    image1=test_pairs[i][1]
    image2=test_pairs[i][2]
    x0 = tface1[a.index(str(test_pairs[i][1])),:,:]
    x1 = tface1[a.index(str(test_pairs[i][2])),:,:]
    if(i%100==0):
        print(i)
    face = x0-x1
    face = np.multiply(face,face)
    face=face.ravel()
    inp = np.zeros((1,32*32))
    inp[0,:]=face
    euc.append(clf.predict(inp))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [85]:
#1 is similar; 0 is different
with open('siamese1.csv','w') as f : 
    f.write('Id,Expected\n')
    for i in range(len(test_pairs)) : 
        ids = test_pairs[i][0]
        temp = euc[i][0]
        f.write(str(ids)+','+str(temp)+'\n')

In [86]:
euc = np.asarray(euc)

In [88]:
(91*3540)/100

3221.4

In [89]:
3540-3221

319

In [93]:
clf.predict(X_train)

array([1, 1, 1, ..., 0, 0, 1])

In [95]:
sum(clf.predict(X_train))

787